In [291]:
import numpy as np
import pandas as pd

importance_matrix = pd.read_excel("../data/feature_importance.xlsx", sheet_name=4)
N = importance_matrix.shape[0]

Judgment_matrix_A = importance_matrix.drop("Unnamed: 0", axis=1).values
importance_matrix.iloc[:,0]

0     NAME_EDUCATION_TYPE
1             CODE_GENDER
2            FLAG_OWN_CAR
3         FLAG_OWN_REALTY
4        AMT_INCOME_TOTAL
5        NAME_INCOME_TYPE
6      NAME_FAMILY_STATUS
7                     age
8              occupation
9                 capital
10         hours-per-week
Name: Unnamed: 0, dtype: object

In [292]:
weights_W = []

for i in range(Judgment_matrix_A.shape[0]):
    weights_W.append(np.power(np.prod(Judgment_matrix_A[i]), 1/N))

std_weights_W = weights_W/np.sum(weights_W)
std_weights_W

array([0.06324513, 0.2670073 , 0.06735867, 0.01720726, 0.01998436,
       0.08033522, 0.18545356, 0.11098794, 0.05629297, 0.02864629,
       0.1034813 ])

In [293]:
np.sum(std_weights_W)

1.0

In [294]:
A_W = []
for i in range(Judgment_matrix_A.shape[0]):
    A_W.append(np.sum(std_weights_W*Judgment_matrix_A[i]))

lambda_max = np.sum(A_W / (N * std_weights_W))
lambda_max

12.379084394285133

In [295]:
CI = (lambda_max - N) / (N - 1)
RI = [0, 0, 0.68, 0.90, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49, 1.51, 1.54, 1.56]
CR = CI/RI[10]
CR

0.0913300923367638

In [296]:
feature_weights = pd.DataFrame({"Feature_name": importance_matrix.iloc[:,0], "Weight": std_weights_W})
feature_weights

,Feature_name,Weight
0,NAME_EDUCATION_TYPE,0.063245
1,CODE_GENDER,0.267007
2,FLAG_OWN_CAR,0.067359
3,FLAG_OWN_REALTY,0.017207
4,AMT_INCOME_TOTAL,0.019984
5,NAME_INCOME_TYPE,0.080335
6,NAME_FAMILY_STATUS,0.185454
7,age,0.110988
8,occupation,0.056293
9,capital,0.028646


In [297]:
feature_weights.to_csv('../data/feature_weights.csv', index=False)

In [298]:
# To calculate the credit score
df = pd.read_csv('../data/final_df_2.csv')
ID = df["ID"]
df.drop(['ID','workclass'], axis=1, inplace=True)
display(df)
df.info()

,NAME_EDUCATION_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,DAYS_EMPLOYED,age,occupation,capital,hours-per-week,credit_decision
0,Higher education,M,Y,Y,427500.0,Working,Civil marriage,-4542,36.332433,Exec-managerial,509.390789,39.454320,1
1,Higher education,M,Y,Y,427500.0,Working,Civil marriage,-4542,36.332433,Exec-managerial,509.390789,39.454320,1
2,Secondary / secondary special,M,Y,Y,112500.0,Working,Married,-1134,38.135474,Other-service,440.541682,39.552563,-1
3,Secondary / secondary special,F,N,Y,270000.0,Commercial associate,Single / not married,-3051,38.135474,Craft-repair,440.541682,39.552563,-1
4,Secondary / secondary special,F,N,Y,270000.0,Commercial associate,Single / not married,-3051,38.135474,Adm-clerical,440.541682,39.552563,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,Secondary / secondary special,M,Y,Y,315000.0,Working,Married,-2420,38.135474,Sales,440.541682,39.552563,-1
36453,Higher education,F,N,Y,157500.0,Commercial associate,Married,-1325,36.332433,Sales,509.390789,39.454320,-1
36454,Higher education,F,N,Y,157500.0,Pensioner,Married,-1325,36.332433,Sales,509.390789,39.454320,-1
36455,Secondary / secondary special,F,N,Y,283500.0,Working,Married,-655,38.135474,Other-service,440.541682,39.552563,-1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36457 entries, 0 to 36456
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NAME_EDUCATION_TYPE  36457 non-null  object 
 1   CODE_GENDER          36457 non-null  object 
 2   FLAG_OWN_CAR         36457 non-null  object 
 3   FLAG_OWN_REALTY      36457 non-null  object 
 4   AMT_INCOME_TOTAL     36457 non-null  float64
 5   NAME_INCOME_TYPE     36457 non-null  object 
 6   NAME_FAMILY_STATUS   36457 non-null  object 
 7   DAYS_EMPLOYED        36457 non-null  int64  
 8   age                  36457 non-null  float64
 9   occupation           36457 non-null  object 
 10  capital              36457 non-null  float64
 11  hours-per-week       36457 non-null  float64
 12  credit_decision      36457 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 3.6+ MB


In [299]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
y = df['credit_decision']
df.drop(['credit_decision'],axis=1,inplace=True)
for col in df.columns:
    if df[col].dtype != 'object':
        df[col] = scaler.fit_transform(df[[col]])
    else:
        df_dummy = pd.get_dummies(df[col],drop_first=True,prefix=col).astype(int)
        df = pd.concat([df,df_dummy],axis=1)
        df.drop(col,axis=1,inplace=True)
df

,AMT_INCOME_TOTAL,DAYS_EMPLOYED,age,capital,hours-per-week,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,CODE_GENDER_M,FLAG_OWN_CAR_Y,...,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Machine-op-inspct,occupation_Other-service,occupation_Prof-specialty,occupation_Sales
0,2.365845,-0.463532,-1.131061,0.672369,-0.462054,1,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
1,2.365845,-0.463532,-1.131061,0.672369,-0.462054,1,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
2,-0.728827,-0.438774,0.315873,-0.244936,0.219806,0,0,1,1,1,...,0,0,0,0,0,0,0,1,0,0
3,0.818509,-0.452700,0.315873,-0.244936,0.219806,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,0
4,0.818509,-0.452700,0.315873,-0.244936,0.219806,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,1.260605,-0.448116,0.315873,-0.244936,0.219806,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
36453,-0.286731,-0.440161,-1.131061,0.672369,-0.462054,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
36454,-0.286731,-0.440161,-1.131061,0.672369,-0.462054,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
36455,0.951138,-0.435294,0.315873,-0.244936,0.219806,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [300]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

df_non_education = df.loc[:, ~df.columns.str.startswith('NAME_EDUCATION_TYPE')]
vif_data = pd.DataFrame()
vif_data["feature"] = df_non_education.columns
vif_data["VIF"] = [variance_inflation_factor(df_non_education.values, i)
                          for i in range(len(df_non_education.columns))]
vif_data.sort_values(by="VIF", ascending=False)

,feature,VIF
8,NAME_INCOME_TYPE_Pensioner,88.166451
1,DAYS_EMPLOYED,77.070595
12,NAME_FAMILY_STATUS_Married,8.412469
3,capital,4.095083
11,NAME_INCOME_TYPE_Working,3.235483
4,hours-per-week,3.226354
7,FLAG_OWN_REALTY_Y,3.014875
14,NAME_FAMILY_STATUS_Single / not married,2.404463
16,occupation_Craft-repair,2.281363
2,age,2.252446


In [301]:
drop_columns = vif_data['feature'][vif_data['VIF'] > 5].tolist()
print(drop_columns)
X = df.drop(drop_columns,axis=1)
display(X.columns)
X

['DAYS_EMPLOYED', 'NAME_INCOME_TYPE_Pensioner', 'NAME_FAMILY_STATUS_Married']


Index(['AMT_INCOME_TOTAL', 'age', 'capital', 'hours-per-week',
       'NAME_EDUCATION_TYPE_Higher education',
       'NAME_EDUCATION_TYPE_Lower secondary',
       'NAME_EDUCATION_TYPE_Secondary / secondary special', 'CODE_GENDER_M',
       'FLAG_OWN_CAR_Y', 'FLAG_OWN_REALTY_Y', 'NAME_INCOME_TYPE_State servant',
       'NAME_INCOME_TYPE_Student', 'NAME_INCOME_TYPE_Working',
       'NAME_FAMILY_STATUS_Separated',
       'NAME_FAMILY_STATUS_Single / not married', 'NAME_FAMILY_STATUS_Widow',
       'occupation_Craft-repair', 'occupation_Exec-managerial',
       'occupation_Farming-fishing', 'occupation_Machine-op-inspct',
       'occupation_Other-service', 'occupation_Prof-specialty',
       'occupation_Sales'],
      dtype='object')

,AMT_INCOME_TOTAL,age,capital,hours-per-week,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,CODE_GENDER_M,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,...,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Machine-op-inspct,occupation_Other-service,occupation_Prof-specialty,occupation_Sales
0,2.365845,-1.131061,0.672369,-0.462054,1,0,0,1,1,1,...,0,0,0,0,1,0,0,0,0,0
1,2.365845,-1.131061,0.672369,-0.462054,1,0,0,1,1,1,...,0,0,0,0,1,0,0,0,0,0
2,-0.728827,0.315873,-0.244936,0.219806,0,0,1,1,1,1,...,0,0,0,0,0,0,0,1,0,0
3,0.818509,0.315873,-0.244936,0.219806,0,0,1,0,0,1,...,0,1,0,1,0,0,0,0,0,0
4,0.818509,0.315873,-0.244936,0.219806,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,1.260605,0.315873,-0.244936,0.219806,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
36453,-0.286731,-1.131061,0.672369,-0.462054,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
36454,-0.286731,-1.131061,0.672369,-0.462054,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
36455,0.951138,0.315873,-0.244936,0.219806,0,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [302]:
weights_continuous = feature_weights.iloc[[4, 7, 9, 10], 1].values
education = np.array([feature_weights.iloc[0, 1]/3 for _ in range(3)])
gender_car_realty = feature_weights.iloc[1:4, 1].values
income_type = np.array([feature_weights.iloc[5, 1]/3 for _ in range(3)])
family_status = np.array([feature_weights.iloc[6, 1]/3 for _ in range(3)])
occupation = np.array([feature_weights.iloc[8, 1]/7 for _ in range(7)])

weights = np.concatenate((weights_continuous, education, gender_car_realty, income_type, family_status, occupation))

In [303]:
credit_scores = np.matmul(X, weights.reshape(-1,1))
credit_scores.columns= ["credit_scores"]
credit_scores

,credit_scores
0,0.300668
1,0.300668
2,0.443697
3,0.175293
4,0.167252
...,...
36452,0.483455
36453,-0.113486
36454,-0.113486
36455,0.142904


In [304]:
scaled_data = (credit_scores.iloc[:, ] - credit_scores.iloc[:, ].min()) / (credit_scores.iloc[:, ].max() - credit_scores.iloc[:, ].min()) * 100
credit_scores["scaled_credit_scores"] = scaled_data
credit_scores["ID"] = ID
credit_scores = credit_scores[["ID", "credit_scores", "scaled_credit_scores"]]
credit_scores

,ID,credit_scores,scaled_credit_scores
0,5008804,0.300668,10.624178
1,5008805,0.300668,10.624178
2,5008806,0.443697,13.901235
3,5008808,0.175293,7.751627
4,5008809,0.167252,7.567374
...,...,...,...
36452,5149828,0.483455,14.812150
36453,5149834,-0.113486,1.135172
36454,5149838,-0.113486,1.135172
36455,5150049,0.142904,7.009539


In [305]:
credit_scores.to_csv("../data/credit_scores_AHP.csv", index=False)